In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
import cv2
from PIL import Image

In [2]:
DCM_PATH = ''
PNG_PATH = ''
MDL_PATH =  ''

In [3]:
# df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
df = pd.read_csv(os.path.join(DCM_PATH, 'train_labels.csv'))
df

,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1
...,...,...
580,1005,1
581,1007,1
582,1008,1
583,1009,0


In [4]:
# data_dir = '../input/rsna-miccai-png/train'
data_dir = os.path.join(PNG_PATH, 'train')
patients = sorted(os.listdir(data_dir))
patients[:5]

['00000', '00002', '00003', '00005', '00006']

In [5]:
# data_dir = '../input/rsna-miccai-png/test'
data_dir = os.path.join(PNG_PATH, 'test')
patients_test = sorted(os.listdir(data_dir))
patients_test[:5]

['00001', '00013', '00015', '00027', '00037']

In [6]:
id = []
for i in patients:
    id.append(int(i))
id[:5]

[0, 2, 3, 5, 6]

In [7]:
train_dataset = pd.DataFrame({'path' : glob(os.path.join(PNG_PATH, 'train/00000/FLAIR/*.png')),
                              'label' : 1})

for i, patient in enumerate(patients[1:]):
    data_slice = pd.DataFrame({'path' : glob(os.path.join(PNG_PATH, 'train/' + patient +'/FLAIR/*.png')),
                               'label' : df['MGMT_value'][i+1]})
    train_dataset = pd.concat([train_dataset, data_slice])
    
train_dataset = train_dataset.reset_index(drop = True)
train_dataset['label'] = train_dataset['label'].astype(str)
train_dataset

,path,label
0,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,1
1,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,1
2,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,1
3,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,1
4,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,1
...,...,...
53677,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,0
53678,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,0
53679,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,0
53680,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...,0


In [8]:
test_dataset = pd.DataFrame({'path' : glob(os.path.join(PNG_PATH, 'test/00001/FLAIR/*.png'))})

for i, patient in enumerate(patients_test[1:]):
    data_slice = pd.DataFrame({'path' : glob(os.path.join(PNG_PATH, 'test/' + patient +'/FLAIR/*.png'))})
    test_dataset = pd.concat([test_dataset, data_slice])
    
test_dataset = test_dataset.reset_index(drop = True)

test_dataset

,path
0,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
1,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
2,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
3,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
4,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
...,...
7921,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
7922,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
7923,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...
7924,/mnt/nas03/RSNA_2021/Brain_Tumor_Classificatio...


In [9]:
from sklearn.model_selection import train_test_split

x_train, x_valid = train_test_split(train_dataset, stratify = train_dataset['label'], random_state = 42, test_size = 0.2)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

from keras import *
from keras.layers import *
from tensorflow.keras.applications import EfficientNetB0
from keras.callbacks import *

idg = ImageDataGenerator(horizontal_flip = True)
idg2 = ImageDataGenerator()

train_dataset = idg.flow_from_dataframe(x_train, x_col = 'path', y_col = 'label', target_size=(150, 150), batch_size = 64)
valid_dataset = idg2.flow_from_dataframe(x_valid, x_col = 'path', y_col = 'label', target_size=(150, 150), batch_size = 64)

efn = EfficientNetB0(include_top = False, pooling = 'avg', input_shape=(150, 150, 3),
                     weights='')

es = EarlyStopping(patience = 2, restore_best_weights = True)
rl = ReduceLROnPlateau(patience = 1, factor = 0.2, verbose = 1)

model = Sequential()
model.add(efn)
model.add(Dense(2, activation = 'softmax'))

model.compile(metrics = ['acc'], loss = 'categorical_crossentropy', optimizer = 'adam')

model.fit(train_dataset, validation_data = valid_dataset, epochs = 1, callbacks = [es, rl])

test_generator = idg2.flow_from_dataframe(test_dataset, x_col = 'path', y_col = None, target_size = (150, 150), batch_size = 64, class_mode = None, shuffle = False)

result = model.predict(test_generator, verbose = True, workers = 2)

Found 42945 validated image filenames belonging to 2 classes.
Found 10737 validated image filenames belonging to 2 classes.
672/672 [==============================] - 108s 153ms/step - loss: 0.6267 - acc: 0.6398 - val_loss: 0.6365 - val_acc: 0.6127
Found 7926 validated image filenames.
124/124 [==============================] - 6s 43ms/step


In [11]:
result

array([[0.5855852 , 0.41441482],
       [0.6660884 , 0.33391166],
       [0.70057404, 0.2994259 ],
       ...,
       [0.6799016 , 0.3200984 ],
       [0.6711853 , 0.32881466],
       [0.74973595, 0.25026405]], dtype=float32)

In [12]:
final_result = []
i = 0
for patient in patients_test:
    length = len(glob(os.path.join(PNG_PATH, 'test/' + patient +'/FLAIR/*.png')))
    final_result.append(result[i:i+length, 1].sum()/length)
    i += length
    
final_result

[0.42804718017578125,
 0.30819189889090404,
 0.441809786690606,
 0.5116830883604108,
 0.3714850519744443,
 0.5151924505466368,
 0.4618559413486057,
 0.4106723221716602,
 0.5118388654498444,
 0.5300032021342844,
 0.37083864212036133,
 0.3464262882868449,
 0.3770566213698614,
 0.40990141459873747,
 0.40412613323756624,
 0.3641828190196644,
 0.3765393037062425,
 0.43517157610724955,
 0.40386204097581946,
 0.4608940124511719,
 0.39348487854003905,
 0.3432445526123047,
 0.4231462097167969,
 0.38117655118306476,
 0.3827667642146983,
 0.4740547339121501,
 0.38330674171447754,
 0.4315720848415209,
 0.3920371386469627,
 0.34736436063593085,
 0.45054986741807723,
 0.36032748222351074,
 0.3826110109369806,
 0.42780563354492185,
 0.3522523520649343,
 0.38426407708062066,
 0.3810793694029463,
 0.45603967220225233,
 0.3988810181617737,
 0.43123793601989746,
 0.4569863712086397,
 0.3801081231299867,
 0.4243117799150183,
 0.42280209318120426,
 0.4154274410671658,
 0.3972301897795304,
 0.40300136566162

In [13]:
test = pd.read_csv(os.path.join(DCM_PATH, "sample_submission.csv"), index_col="BraTS21ID")
test

,MGMT_value
BraTS21ID,
1,0.5
13,0.5
15,0.5
27,0.5
37,0.5
...,...
826,0.5
829,0.5
833,0.5


In [14]:
test['MGMT_value'] = final_result

In [15]:
test["MGMT_value"].to_csv("submission.csv")

In [18]:
test.head()

,MGMT_value
BraTS21ID,
1,0.428047
13,0.308192
15,0.441810
27,0.511683
37,0.371485
